<a href="https://colab.research.google.com/github/danilomennezes/LLM-PLN/blob/main/LLM_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM - Apresentação LangChain

Apresntação de LangChain para alunos do Curso de Ciencias da Computação da UNESP de São José do Rio Preto.

https://www.langchain.com/

https://python.langchain.com/v0.2/docs/tutorials/llm_chain/

In [ ]:
!pip install langchain
!pip install langchain_openai
!pip install langchain-community
from langchain_openai import OpenAI, ChatOpenAI #importando os pacotes OpenAI dentro da LangChain

import os #bibliota que cria variaveis de ambiente

token = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' #Substitua o 'xxxxxxxxxxxxxxxxxxx' com o seu token, adquirido em https://platform.openai.com/
os.environ['OPENAI_API_KEY'] = token #Inserindo tokem


##Primeiro exemplos


In [ ]:
#modelo Completion - Gerar texto com base em uma entrada fornecida
openai = OpenAI(model_name='gpt-3.5-turbo-instruct') #Instanciando o modelo, através da classe OpenAI dentro da biblioteca de LangChain
response = openai.invoke(input='segunda guerra mundial?',
                         temperature=1,
                         max_tokens=100, ) #funcão invoke faz a chamada do modelo e passa os input e recebe o output
print(response)

In [ ]:
#Modelo chatComletion - Gerar conversa mantendo o contexto
#Classe ChatOpenAI
openai = ChatOpenAI(model_name='gpt-3.5-turbo')
messages = [
    {"role": "system", "content": "Você é um assistente especialista em história."},
    {"role": "user", "content": "Segunda guerra mundial??"}
]
response = openai.invoke(messages)
print(response.content)

##Memory
A memória em LangChain é uma ferramenta para gerenciar e manter o contexto em aplicações baseadas em modelos de linguagem. Permite que o sistema armazene e recupere informações relevantes ao longo de interações, melhorando a coesão e a personalização das respostas.

Ainda permite consultas a perguntas repetidas, buscando em sua memória para evitar custos com a API



In [ ]:
from langchain_openai import OpenAI
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache

In [ ]:
#Exemplo 1 - Guardando na memória perguntas repetitivas afim de evitar custos com consultas ao GPT-3.5
openai = OpenAI(model_name='gpt-3.5-turbo-instruct', temperature =  1, max_tokens=50)
set_llm_cache(InMemoryCache())

In [ ]:
resposta_1 = openai.invoke('Como fazer uma limonada')
print("Primeira Resposta: ", resposta_1)

In [ ]:
# Ao repetir a pergunta, o modelo não faz a consulta ao GPT-3.5, pois a mesma já se encontra em sua memória
resposta_2 = openai.invoke('Como fazer uma limonada')
print("Segunda Resposta:", resposta_2)

In [ ]:
#Exemplo 2 - Guardando a conversa em memória para dar sentio e contexto
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [ ]:
openai = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.0,max_tokens = 150)
memory = ConversationBufferMemory() #Instância para armazenar e recuperar o histórico da conversa.

In [ ]:
#Cria uma instância de ConversationChain, configurada com o modelo de linguagem openai e o mecanismo de memória memory.
conversation = ConversationChain( llm=openai,  memory = memory, verbose= True)

In [ ]:
conversation.predict(input="Olá! como fazer uma limonada?")

In [ ]:
conversation.predict(input='mas vai ficar muito azeda?' )

In [ ]:
memory.buffer #Verificando a memória

##Template
Um template em LangChain é uma forma de definir um padrão ou esqueleto para a criação de texto, que pode ser preenchido com dados variáveis.

Ajuda a criar prompts ou outras partes de texto de maneira mais estruturada e reutilizável, permitindo que você defina como as informações devem ser organizadas e apresentadas.




In [ ]:
from langchain_openai import ChatOpenAI,OpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

In [ ]:
#Define uma variavel com o template, passando os campos variaveis
template = '''
           Você é um analista do mercado financeiro.
           Faça um breve analise dos seguintes itens {itens} da empresa "{empresa}"
           Leve em consideração o período {periodo}.
'''

In [ ]:
Prompt_Template = PromptTemplate.from_template(template=template) #Instancia o modelo

In [ ]:
#Instânciando o Prompt Template
prompt = Prompt_Template.format(
    empresa='ITUB4',
    periodo='Primeiro trimestre de 2024',
    itens='PVP e IBITDA /n preco sob'
)
print("ChatGPT\n", prompt)

In [ ]:
#Chamar o modelo LangChain chatgpt
openai = OpenAI(model_name='gpt-3.5-turbo-instruct', max_tokens=2000)

response = openai.invoke(prompt)
print("ChatGPT\n", response)

##Chains - Router Chains
Sequenciar chamadas, que serão utilizadas de maneira encadeada (Definir etapas)

Sequencial chains: Utilizam sequencias para um atendimento, podendo fazer uma consulta no manual do produto e outra consulta no registro de atendimentos, enviar ambas ao GPT e criar uma resposta final.

Router chains:Definem rotas que serão executada de acordo com o inout de entrada


In [ ]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


In [ ]:
#Instanciando o modelo da OpenAI
openai = OpenAI(model_name='gpt-3.5-turbo-instruct', temperature=0)

Desenvolveremos um modelo, que será capaz de classificar uma pergunta e logo após acionar o template especifico.

In [ ]:
template_clacificar = (
    PromptTemplate.from_template(
        """    Classifique a pergunta do usuário em uma das seguintes categorias:
              - medico
              - juridico
              - financeiro
              - tecnologia
              - Outras Informações

              Pergunta: {query}
              Classificação:
              """ )
    | openai
    | StrOutputParser() #garante que a saida será uma string
)

In [ ]:
#Exemplo fazendo a chamada em GPT para classificar
item_classificado = template_clacificar.invoke({"query": "processo de execução fiscal"})
print(item_classificado)

In [ ]:
print(template_clacificar)

In [ ]:
# Chains sequencial para cada topico
chain_medico = PromptTemplate.from_template(
    """
    Você é um médico.
    Comece a resposta com "Ola, sou um Médico" e logo após um resumo da pergunta do usuario.
    Responda à pergunta do usuário:
    Pergunta: {query}
    Resposta:
    """
) | openai
chain_advogado = PromptTemplate.from_template(
    """
    Você é um advogado.
    Comece a resposta com "Ola, sou um Advogado" e logo após um resumo da pergunta do usuario.
    Responda à pergunta do usuário:
    Pergunta: {query}
    Resposta:
    """
) | openai
chain_analista_financeiro = PromptTemplate.from_template(
    """
    Você é um analista financeiro.
    Comece a resposta com "Ola, sou um Analista Financeiro" e logo após um resumo da pergunta do usuario.
    Responda à pergunta do usuário:
    Pergunta: {query}
    Resposta:
    """
) | openai

chain_analista_ti = PromptTemplate.from_template(
    """
    Você é um analista de TI.
    Comece a resposta com "Ola, sou um Analista de TI" e logo após um resumo da pergunta do usuario.
    Ajude o usuário com seu problema técnico.
    Pergunta: {query}
    Resposta:
    """
) | openai

chain_outras_informacoes = PromptTemplate.from_template(
    """
    Você é um assistente de informações gerais.
    Comece a resposta com "Ola, sou um analista Geral" e logo após um resumo da pergunta do usuario.
    Forneça informações ao usuário sobre sua pergunta.
    Pergunta: {query}
    Resposta:
    """
) | openai

In [ ]:
# Função de roteamento
def route(info):
    topic = info["topic"].lower()
    if "financeiro" in topic:
        return chain_analista_financeiro
    elif "medico" in topic:
        return chain_medico
    elif "juridico" in topic:
        return chain_advogado
    elif "tecnologia" in topic:
        return chain_analista_ti
    else:
        return chain_outras_informacoes

In [ ]:
Input = input("Ola, o que deseja ?")

In [ ]:
# Perguntas
item_classificado = template_clacificar.invoke({"query": Input })
prompt = route({"topic": item_classificado})

response = prompt.invoke({"query":Input})
print(response)

In [ ]:
print(prompt)

##Tools

Tools são ferramentas úteis que auxiliam os modelos de linguagem, permitindo integrações.

Estas ferramentas realizam determinadas ativadades, como buscar dados na internet, Wikipia, fazer analise ou calculos



In [ ]:
!pip install langchain_experimental
from langchain_experimental.utilities import PythonREPL #ferramenta python
!pip install duckduckgo-search
from langchain_community.tools import DuckDuckGoSearchRun,WikipediaQueryRun #busca web
!pip install wikipedia
from langchain_community.utilities import WikipediaAPIWrapper #busca Wikipedia

In [ ]:
#Exemplo1 - Exemplo isolados
#fazendo a requisão para Python de calculos
python_repl = PythonREPL()
result = python_repl.run("print(50/10)") #utiliza a instancia do objto run
print(result)

In [ ]:
#fazendo a requisão para duckduckgo de busca na WEB
ddg_search = DuckDuckGoSearchRun()
search_results = ddg_search.run("segunda guerra mundial? traduza para o portugues")
print(search_results)

In [ ]:
#fazendo a requisão para Wikipedia
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
query = "segunda guerra mundial"
wikipedia_results = wikipedia.run(query)
print("Wikipedia:", wikipedia_results)

##Agentes

São componentes que utilizam modelos de linguagem para tomar decisões, executar ações e gerar respostas com base em uma série de condições e regras. Eles são projetados para atuar como intermediários entre o modelo de linguagem e outras partes do sistema, como APIs, bancos de dados ou interfaces de usuário.



In [ ]:
from langchain_experimental.agents.agent_toolkits import create_python_agent #Agentes python
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.prompts import PromptTemplate

In [ ]:
openai = OpenAI(model_name='gpt-3.5-turbo-instruct', temperature=0, max_tokens=200)

In [ ]:
#Instanciando o DuckDuckGo
ddg_search = DuckDuckGoSearchRun()

In [ ]:
#Criando o Agente (create_python_agent), em llm passamos o openai, em ferramenta passamos a busca na web DuckDuckGo, verbose true para debugar
agent_executor = create_python_agent(llm=openai, tool=ddg_search, verbose=True)

#Criando o Template de Prompt passando a variavel query
prompt_template = PromptTemplate( input_variables=["query"],
                                  template=""" Pesquise na web sobre {query} e forneça um resumo. """)

In [ ]:
query = "segunda guerra mundial"
prompt = prompt_template.format(query=query)
print(f"prompt:\n{prompt}")

In [ ]:
#Executar o agente, passando o prompt
response = agent_executor.invoke(prompt)
#Imprimindo a resposta
print(f"return: \n", response['output'])

##React Agents

React Agentes é uma extensão dos agentes simples, introduzindo a capacidade de reagir e adaptar-se a mudanças, consegue determinar qual a melhor maneira de atender ao prompt, podendo ser capaz de utilizar diversas tools

In [ ]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

In [ ]:
openai = OpenAI(model_name='gpt-3.5-turbo-instruct', temperature=0, max_tokens=2000)

Desenvolveremos um modelo, que será capaz de entender uma pergunta e ter autonomia para buscar a emlhor forma de responder.

Neste exemplo o modelo utilizara uma ferramenta para cálculo e outra para buscar sugestões na internet.


In [ ]:
#Prompt
prompt = ''' Sempre responda, informando o saldo, fazendo um resumo da pergunta e passando dicas de finanças.
             Pergunta: {pergunta}'''
prompt_template = PromptTemplate.from_template(prompt)
print(prompt_template)

In [ ]:
#instruções - Template do Hub da LangChain, com instruções de como o agente deve se comportar
react_instructions  = hub.pull('hwchase17/react')

In [ ]:
#Criando as Tools - Ferramentas que serão utilizadas para que o nosso modelo possa responder a pergunta do usuário.
#Passar regras claras para o prompt. Um prompt bem definido pode diminuir a possibilidade de alucinação do GPT.

# Ferramenta calculo Python REPL
python_repl = PythonREPLTool() #ferramenta para executar código python (podendo fazer calculo)
python_repl_tool = Tool( name='Python REPL',
                         func=python_repl.run, ##run para executar esta ferramenta
                         description='''Para calculos, utilize esta ferramenta com código python.'''

)

# Ferramenta  busca DuckDuckGo
search = DuckDuckGoSearchRun() #ferramenta para fazer buscas na WEB
duckduckgo_tool = Tool(
    name='Busca DuckDuckGo',
    func=search.run,
    description='''Para encontrar informacoes na internet, utilize esta ferramenta'''
)

In [ ]:
#Instanciando o AgentExecutor
agent_executor = AgentExecutor(
    agent=create_react_agent(openai, [python_repl_tool, duckduckgo_tool], react_instructions),
    tools=[python_repl_tool, duckduckgo_tool],
    verbose=True,
    handle_parsing_errors=True, #controlar como o agente deve lidar com erros
    max_iterations=10 #define o número máximo de iterações que o agente pode realizar, para evitar loop
)
print(agent_executor)

In [ ]:
prompt_pergunta = """
Minha renda mensal é de 7500, o total de minhas despesas mensais é R$2500 mais 1000 de aluguel.
Poderia me dar algumas dicas de investimento?
"""

In [ ]:
output = agent_executor.invoke({
    'input': prompt_template.format(pergunta=prompt_pergunta)
})

In [ ]:
print(output['input'])

In [ ]:
print(output['output'])